# Ejemplos de Programación Orientada a Objetos Involucrando el método `Super()` con `__Init__()`

Mediante la noción de herencia en Python vamos a tratar de materializar en código el siguiente esquema a modo de ejemplo explicativo:

                            Object
                              |
                             ABC
                              |
                            Shape
                          /       \
                       Circle   Triangle 
                          \        / 
                           \      /
                             Cone

In [ ]:
from abc import ABC, abstractmethod
from math import pi, sqrt

In [ ]:
class Shape(ABC):
    """
    Explicitamos que Shape cuenta con dos funciones sin código, código que será materializado en la
    herencia. perimeter (resp. area) representa un valor asociado a cada instancia de Shape que estemos 
    manipulando pero con la unidad de medida elevada a la dimensión de la instancia menos uno (resp. 
    de la instancia). Por ejemplo, si pensamos en instancias de dimensión 2 (tríangulos, círculos, etc.), 
    perimeter tendrá salida con unidades u^1 y area tendrá una salida con unidades u^2; pero si 
    pensamos en instancias de dimensión 3 (conos, etc.), perimeter tendrá salida con unidades u^2 
    (área lateral total) y area tendrá una salida con unidades u^3 (volumen). Si alguien siente recelo
    de esto, pues que renombre `perimeter` y `area` a un nombre menos sugerente 
    respecto a conceptos geométricos, p.e. que renombre "perimeter" a "m" y "area" a "n".
    """
    @abstractmethod
    def perimeter(self):
        pass
    @abstractmethod
    def area(self):
        pass

In [ ]:
class Circle(Shape):
    # Initialize the Circle object with a given radius
    def __init__(self, radius, **kwargs):
        self.radius = radius
        super().__init__(**kwargs) # reenvía todos los argumentos no utilizados

    # Calculate and return the area of the circle using the formula: π * r^2
    def area(self):
        return pi * self.radius**2

    # Calculate and return the perimeter of the circle using the formula: 2π * r
    def perimeter(self):
        return 2 * pi * self.radius

    def __repr__(self):
        return f'Circle({self.radius})'

    def __str__(self):
        r0 = self.radius
        r1, r2 = self.area(), self.perimeter()
        return "Circle({0},{1} u^2,{2} u)".format(r0,r1,r2)

In [ ]:
r = 7

In [ ]:
circle = Circle(r)

In [ ]:
circle

In [ ]:
circle.area()

In [ ]:
circle.perimeter()

In [ ]:
print(circle)

In [ ]:
class Triangle(Shape):
    """
    Clase de los triángulo para la formación de los mismos,
    de ser posible, y la obtención de su área y perímetro.
    """
    # decides whether a triad of numbers forms a triangle or not
    def _es_triangulo(self, a, b, c):
        lst = [a,b,c]
        lst.sort()
        longer = lst.pop()
        return sum(lst) > longer

    # Calculates the semi-perimeter of a triangle
    def _semiperimeter(self):
        return (self.side1 + self.side2 + self.side3) / 2

    # Initialize the Triangle object with three side lengths if possible
    def __init__(self, side1, side2, side3, **kwargs):
        if self._es_triangulo(side1,side2,side3):
            self.side1 = side1
            self.side2 = side2
            self.side3 = side3
        else:
            self.side1, self.side2, self.side3 = 0, 0, 0

        super().__init__(**kwargs)

    # Calculate and return the area of the triangle using the formula: 0.5 * base * height
    def area(self):
        sp1, sp2, sp3 = self.side1, self.side2, self.side3
        sp = self._semiperimeter()
        return sqrt(sp*(sp-sp1)*(sp-sp2)*(sp-sp3))

    # Calculate and return the perimeter of the triangle by adding the lengths of its three sides
    def perimeter(self):
        return self.side1 + self.side2 + self.side3

    def __repr__(self):
        return f'Triangle({self.side1},{self.side2},{self.side3})'

    def __str__(self):
        s0,s1,s2 = self.side1, self.side2, self.side3
        s3, s4 = self.area(), self.perimeter()
        return "T({0},{1},{2},{3} u^2,{4} u)".format(s0,s1,s2,s3,s4)

El siguiente ejemplo es un ejemplo no "sano" de triángulo, de hecho no determina un triángulo y todo lo esperable de él va a fracasar, siendo presentados valores absurdos. Incluso pudimos optar por dar la voz de alarma más sonoramente con él haciendo que fuesen elevados errores, que habríamos de capturar constantemente por mor de que no se viese frustrado el flujo del programa.

In [ ]:
c = Triangle(1,2,3)

In [ ]:
print(c)

In [ ]:
c.side1

In [ ]:
c.area()

In [ ]:
c.perimeter()

Sin embargo, el siguiente sí es un triángulo "sano".

In [ ]:
d = Triangle (2,2,3)

In [ ]:
d

In [ ]:
repr(d)

In [ ]:
d.side3

In [ ]:
d.perimeter()

In [ ]:
d._semiperimeter()

In [ ]:
d.area()

In [ ]:
print(d)

In [ ]:
print(repr(d))

In [ ]:
class Cone(Triangle, Circle):
    """
    Clase Cone para definir y utilizar ramas de cono truncadas
    por un plano horizontal. Una tal rama es la revolución de un
    triángulo rectángulo, por lo que puede ser considerada heredera
    de la clase Triangle. Esa rama también viene dada por un círculo
    a cierta distancia del origen cónico.
    """
    def _pitagoras(self,side1,side2):
        return sqrt(side1**2+side2**2)

    def __init__(self, radius, height, **kwargs):
         generatrix = self._pitagoras(radius,height)
         super().__init__(side1=radius, side2=height, side3=generatrix, radius=radius, **kwargs)
                                    # Pasa todos los argumentos en un esquema clave=valor
                                    # para evitar problemas con
                                    # argumentos posicionales y el orden
                                    # de las clases padre.
                                    # Dejamos preparado para heredar de Cone
    def lateral_area(self):
        return pi * self.radius * self.side3

    def perimeter(self):
        """
        En una generalización "audaz" vamos a considerar que al aumentar de
        dimensión el perímetro (que tiene dimensión uno menos) se materializa 
        en el área total de la rama de cono truncado por un plano paralelo al XY.
        """
        return self.lateral_area() + Circle.area(self)

    def area (self):
        """
        En una generalización "audaz" vamos a considerar que al aumentar de
        dimensión el área (que tiene la dimensión de la figura) se materializa 
        en el volumen de la rama de cono truncado por un plano paralelo al XY.
        """
        return Circle.area(self) * self.side2 / 3

    def __repr__(self):
        return f'Cone({self.radius},{self.side2})'

#    # esta versión de __str__ es sugerente para la informática pero poco significativa en matemáticas
#    def __str__(self):
#           return Circle.__str__(self) + ", " + Triangle.__str__(self)

    def __str__(self):
        s0,s1 = self.radius, self.side2
        s3, s4 = self.perimeter(), self.area()
        return "T({0},{1},{2} u^2,{3} u^3)".format(s0,s1,s3,s4)

## Prueba

In [ ]:
cono1 = Cone(4,3)

In [ ]:
cono1

In [ ]:
cono1.lateral_area()

In [ ]:
cono1.perimeter()

In [ ]:
cono1.area()

In [ ]:
cono1

In [ ]:
print(cono1)

In [ ]:
cono2=Cone(8,18)

In [ ]:
cono2

In [ ]:
print(cono2)

## Obtención de Información sobre Clases

In [ ]:
Cone.__name__

In [ ]:
Cone.__doc__

In [ ]:
print(Cone.__doc__)

In [ ]:
dir(Cone)

In [ ]:
Cone.__dict__

In [ ]:
Cone.__module__

In [ ]:
Cone.__bases__

In [ ]:
print("MRO:", Cone.__mro__)

## Ejercicios

1. Escribar el código para una clase que simule con plena funcionalidad el comportamiento de los números racionales.

## Bibliografía y Enlaces

Para ampliar esta información puede consultar:

* [abc — Abstract Base Classes](https://docs.python.org/es/3.13/library/abc.html#abc.abstractmethod)
* [Calling parent class __init__ with multiple inheritance, what's the right way?](https://stackoverflow.com/questions/9575409/calling-parent-class-init-with-multiple-inheritance-whats-the-right-way)
* [Decoradores en Python: ¿Qué son y cómo funcionan?](https://www.programaenpython.com/avanzado/decoradores-en-python/)
* [OOPs Concepts in Python](https://www.geekster.in/articles/oops-concepts-in-python/)
* [Python Object-Oriented Programming](https://www.w3resource.com/python-exercises/oop/python-oop-exercise-4.php)
* [Python `Super()` with `__Init__()` Method](https://www.geeksforgeeks.org/python-super-with-__init__-method/)
* Ramalho, L.; [Fluent Python](https://github.com/WeitaoZhu/Python/blob/master/Fluent.Python.2nd.Edition.(z-lib.org).pdf). O'Really, 2015.
* Sweigart, A.; [Automate the Boring Stuff with Python](https://automatetheboringstuff.com/). No Starch Press, [2015](https://archive.org/details/Automate-Boring-Stuff-Python-2nd/page/n7/mode/2up).
* [Understanding Python `super()` with `__init__()` methods](https://stackoverflow.com/questions/576169/understanding-python-super-with-init-methods)